In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Train.csv")

In [2]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(df)

RobustScaler()

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_new= pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_new['class']=df['class']
df=df_new

In [4]:
variances = df.var()
columns_to_transforms = variances[variances < 1e-3].index
df[columns_to_transforms] = df[columns_to_transforms].apply(np.sqrt)

In [6]:
variances = df.var()
columns_to_transform = variances[variances < 1e-2].index
df[columns_to_transform] = df[columns_to_transform]*10
df.var()
df['class']=df_new['class']

In [7]:
X = df.drop(['class'], axis=1)
y = df['class']
def z_score_normalize(df):
  result = df.copy()
  for col in df.select_dtypes(include=[np.number]):
    result[col] = (df[col] - df[col].mean()) / df[col].std(ddof=0)
  return result
X=z_score_normalize(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.losses import Hinge

2024-06-20 17:34:41.116556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:34:41.116724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:34:41.275468: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
model=tf.keras.Sequential([
    layers.Dense(28, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(28, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(16, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(8, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    validation_split=0.1, 
    callbacks=[early_stopping],
    batch_size=32,
    verbose=1
)

Epoch 1/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.5730 - loss: 0.7145 - val_accuracy: 0.6452 - val_loss: 0.6404
Epoch 2/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6374 - loss: 0.6421 - val_accuracy: 0.6512 - val_loss: 0.6275
Epoch 3/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6460 - loss: 0.6322 - val_accuracy: 0.6534 - val_loss: 0.6249
Epoch 4/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6534 - loss: 0.6270 - val_accuracy: 0.6542 - val_loss: 0.6232
Epoch 5/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6495 - loss: 0.6263 - val_accuracy: 0.6564 - val_loss: 0.6211
Epoch 6/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6587 - loss: 0.6218 - val_accuracy: 0.6550 - val_loss: 0.6219
Epoch 7/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6525 - loss: 0.6232 - val_accuracy: 0.6599 - val_loss: 0.6207
Epoch 8/100
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6538 - loss: 0

In [13]:
import math
import matplotlib.pyplot as plt
def f1(t):
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob >t).astype(int)

# Calculate F1 Score
    f1 = f1_score(y_test, y_pred)
    return f1
f1(0.37)

128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


0.6978542431857723

In [14]:
test_data = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Test.csv")
Index=test_data['Index']
test_data.drop(['Index'],axis=1,inplace=True)

scaler = MinMaxScaler()
test_data= pd.DataFrame(scaler.fit_transform(test_data), columns=test_data.columns)

test_data[columns_to_transforms] = test_data[columns_to_transforms].apply(np.sqrt)

test_data[columns_to_transform] = test_data[columns_to_transform]*10

test_data=z_score_normalize(test_data)

y_pred_prob = model.predict(test_data)
y_pred = (y_pred_prob>0.37).astype(int)
y_pred

547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [15]:
y_test=[]
for i in range(len(y_pred)):
    y_test.append(y_pred[i][0])
y_test=np.array(y_test)

In [16]:
result = pd.DataFrame({
    'Index': Index,
    'class': y_test
})

In [17]:
result

,Index,class
0,40977,1
1,24157,0
2,56238,1
3,55411,1
4,53175,1
...,...,...
17471,17197,1
17472,14094,1
17473,34598,1
17474,17845,0


In [18]:
result.to_csv("submission.csv", index=False) #(for making submission file)